Judul
Pintu tol otomatis "Tol JORR (Jakarta Outer Ringroad)"

Kamus

LIST:

List of String: self.gates
List of Mixed (String and Float): path, new_path
List of Tuples (containing paths and distances): jalur_terdekat, all_possible_paths
List of String: visited

DICTIONARY:

Dictionary of Tuple(String, String) to Float: self.lines
Dictionary of Integer to Dictionary(String to Integer): self.tarif_golongan
Dictionary of Integer to String: self.bank_list
Dictionary of String to List(Tuple(String, Float)): graph
Dictionary of String to Integer/Float: tarif

FUNGSI:
init, find_paths_via_neighbors, find_shortest_path_via_neighbor, dfs_path, hitung_tarif, pembayaran_alternatif, proses_pembayaran_qris, proses_virtual_account, cetak_struk, proses_transaksi


Menjalankan Program

In [ ]:
# Program  Pintu tol otomatis "Tol JORR (Jakarta Outer Ringroad)"
# Program yang menjalankan pintu tol otomatis untuk rute Lingkar Luar Jakarta

"""
Kamus

LIST:

List of String: self.gates
List of Mixed (String and Float): path, new_path
List of Tuples (containing paths and distances): jalur_terdekat, all_possible_paths
List of String: visited

DICTIONARY:

Dictionary of Tuple(String, String) to Float: self.lines
Dictionary of Integer to Dictionary(String to Integer): self.tarif_golongan
Dictionary of Integer to String: self.bank_list
Dictionary of String to List(Tuple(String, Float)): graph
Dictionary of String to Integer/Float: tarif

FUNGSI:

init, find_paths_via_neighbors, find_shortest_path_via_neighbor, dfs_path, hitung_tarif, pembayaran_alternatif, proses_pembayaran_qris, proses_virtual_account

PROSEDUR:

cetak_struk, proses_transaksi

CLASS:

TollGateSystem
"""

from collections import defaultdict
import random

gate_open = """
        ╔════════════════════════════════╗
        ║    PINTU TOL TERBUKA
        ║    Silakan melintas...         ════
        ╚════════════════════════════════╝
        """
gate_close = """
        ╔════════════════════════════════╗
        ║    PINTU TOL TERTUTUP          ║
        ║    Tempelkan kartu e-toll      ║
        ╚════════════════════════════════╝
        """
class TollGateSystem:
    def __init__(self):
        # Gerbang tol JORR
        self.gates = [
            'PURI INDAH', 'CAKUNG', 'SERPONG', 'JATI WARNA', 'JATI ASIH',
            'CIKUNIR', 'TAMAN MINI', 'SOEKARNO-HATTA', 'ROROTAN',
            'CIMANGGIS', 'KUNCIRAN', 'DEPOK', 'PANTAI INDAH KAPUK',
            'PONDOK PINANG', 'TANJUNG PRIOK', 'CIBITUNG', 'CINERE', 'PONDOK INDAH'
        ]

        # Rute antar gerbang tol
        self.lines = {
            ('SOEKARNO-HATTA', 'KUNCIRAN'): 15.2,
            ('SOEKARNO-HATTA', 'PANTAI INDAH KAPUK'): 18.1,
            ('PANTAI INDAH KAPUK', 'PURI INDAH'): 9.7,
            ('KUNCIRAN', 'SERPONG'): 11.2,
            ('KUNCIRAN', 'PURI INDAH'): 14.19,
            ('SERPONG', 'CINERE'): 10.1,
            ('SERPONG', 'PONDOK INDAH'): 12.5,
            ('PURI INDAH', 'PONDOK INDAH'): 7,
            ('PONDOK INDAH', 'PONDOK PINANG'): 2.5,
            ('PONDOK PINANG', 'TAMAN MINI'): 17.7,
            ('PONDOK PINANG', 'CINERE'): 23.5,
            ('CINERE', 'CIMANGGIS'): 14.4,
            ('CINERE', 'DEPOK'): 13,
            ('JATI WARNA', 'JATI ASIH'): 6.3,
            ('JATI ASIH', 'CIKUNIR'): 5.7,
            ('CIKUNIR', 'CAKUNG'): 8.3,
            ('CIBITUNG', 'TANJUNG PRIOK'): 34.7,
            ('CIBITUNG', 'ROROTAN'): 33.9,
            ('ROROTAN', 'TANJUNG PRIOK'): 12.1,
            ('ROROTAN', 'CAKUNG'): 4.6,
            ('TAMAN MINI', 'CIMANGGIS'): 15,
            ('TAMAN MINI', 'CINERE'): 19,
            ('TAMAN MINI', 'JATI WARNA'): 7.1,
            ('CIBITUNG', 'CIMANGGIS'): 31,
            ('CIBITUNG', 'CIKUNIR'): 19,
        }

        # Tarif berdasarkan golongan kendaraan
        self.tarif_golongan = {
            1: {'10 km pertama': 17000, 'per_km': 1500},  # Golongan I
            2: {'10 km pertama': 25000, 'per_km': 2000},  # Golongan II
            3: {'10 km pertama': 25000, 'per_km': 2500},  # Golongan III
            4: {'10 km pertama': 33500, 'per_km': 3000},  # Golongan IV
            5: {'10 km pertama': 33500, 'per_km': 3500}   # Golongan V
        }

        # List bank untuk virtual account
        self.bank_list = {
            1: 'BCA',
            2: 'MANDIRI',
            3: 'BNI',
            4: 'BRI',
            5: 'CIMB NIAGA'
        }

    def find_paths_via_neighbors(self, start, end):
        # Buat graph dari lines
        graph = defaultdict(list)
        for (src, dest), distance in self.lines.items():
            graph[src].append((dest, distance))
            graph[dest].append((src, distance))  # Untuk graph tidak berarah

        # Dapatkan tetangga langsung dari titik awal
        start_neighbors = [neighbor for neighbor, _ in graph[start]]

        # Fungsi rekursif untuk mencari jalur melalui tetangga tertentu
        def find_shortest_path_via_neighbor(neighbor):
            def dfs_path(current, end, path, total_distance, visited):
                if current == end:
                    return [(path, total_distance)]

                all_possible_paths = []

                for next_neighbor, edge_distance in graph[current]:
                    if next_neighbor not in visited:
                        new_path = path + [next_neighbor]
                        new_distance = total_distance + edge_distance

                        new_paths = dfs_path(
                            next_neighbor,
                            end,
                            new_path,
                            new_distance,
                            visited + [next_neighbor]
                        )

                        all_possible_paths.extend(new_paths)

                return all_possible_paths

            # Cari jalur dari tetanggga ke tujuan
            initial_path = [start, neighbor]
            initial_distance = self.lines.get((start, neighbor), self.lines.get((neighbor, start)))

            paths = dfs_path(neighbor, end, initial_path, initial_distance, [start, neighbor])

            # Kembalikan jalur terdekat
            return min(paths, key=lambda x: x[1]) if paths else None

        # Cari jalur tercepat melalui setiap tetangga
        jalur_terdekat = []
        for neighbor in start_neighbors:
            path = find_shortest_path_via_neighbor(neighbor)
            if path:
                jalur_terdekat.append(path)

        return jalur_terdekat

    def hitung_tarif(self, jarak, golongan):
        tarif = self.tarif_golongan.get(golongan, self.tarif_golongan[1])

        # Tarif dasar untuk 10 km pertama
        total_tarif = tarif['10 km pertama']

        # Tambahkan tarif untuk km berikutnya
        if jarak > 10:
            total_tarif += (jarak - 10) * tarif['per_km']

        return int(total_tarif)

    def pembayaran_alternatif(self, tarif):
        """
        Menyediakan metode pembayaran alternatif
        """
        print("\n╔════════════════════════════════╗")
        print("║  METODE PEMBAYARAN ALTERNATIF  ║")
        print("╚════════════════════════════════╝")
        print("1. Bayar dengan QRIS")
        print("2. Transfer Virtual Account Bank")

        while True:
            try:
                pilihan = int(input("Pilih metode pembayaran (1/2): "))
                if pilihan == 1:
                    return self.proses_pembayaran_qris(tarif)
                elif pilihan == 2:
                    return self.proses_virtual_account(tarif)
                else:
                    print("Pilihan tidak valid. Silakan pilih 1 atau 2.")
            except ValueError:
                print("Masukkan harus berupa angka.")

    def proses_pembayaran_qris(self, tarif):
        """
        Simulasi pembayaran menggunakan QRIS
        """
        print("\n╔════════════════════════════════╗")
        print("║        PEMBAYARAN QRIS         ║")
        print("╚════════════════════════════════╝")

        print(f"Total tagihan: Rp {tarif}")
        print("Silakan scan QR Code di mesin pembayaran")

        # Simulasi konfirmasi pembayaran
        konfirmasi = input("Apakah pembayaran QRIS berhasil? (YA/TIDAK): ").upper()

        if konfirmasi == 'YA':
            print("Pembayaran QRIS berhasil!")
            return True
        else:
            print("Pembayaran QRIS gagal.")
            return False

    def proses_virtual_account(self, tarif):
        """
        Simulasi pembayaran menggunakan Virtual Account
        """
        print("\n╔════════════════════════════════╗")
        print("║    PEMBAYARAN VIRTUAL ACCOUNT  ║")
        print("╚════════════════════════════════╝")

        print(f"Total tagihan: Rp {tarif}")

        # Pilih bank
        print("Pilih Bank:")
        for key, bank in self.bank_list.items():
            print(f"{key}. {bank}")

        while True:
            try:
                bank_pilihan = int(input("Pilih bank (1-5): "))
                if bank_pilihan in self.bank_list:
                    bank_terpilih = self.bank_list[bank_pilihan]
                    break
                else:
                    print("Pilihan bank tidak valid.")
            except ValueError:
                print("Masukkan harus berupa angka.")

        # Generate nomor virtual account
        va_number = ''.join([str(random.randint(0,9)) for _ in range(16)])

        print(f"\nNo. Virtual Account {bank_terpilih}: {va_number}")
        print("Silakan transfer sesuai nominal yang tertera")

        # Simulasi konfirmasi pembayaran
        konfirmasi = input("Apakah pembayaran Virtual Account berhasil? (YA/TIDAK): ").upper()

        if konfirmasi == 'YA':
            print("Pembayaran Virtual Account berhasil!")
            return True
        else:
            print("Pembayaran Virtual Account gagal.")
            return False

    def cetak_struk(self, start, end, path, total_distance, jenis_kendaraan, golongan, tarif, saldo):
        """
        Metode untuk mencetak struk transaksi
        """
        print("\n═════════════════════════════════ TRANSAKSI BERHASIL ═════════════════════════════════")
        print(f"Jenis Kendaraan : {jenis_kendaraan.capitalize()}")
        print(f"Golongan        : {golongan}")

        if len(path) < 3:
            print(f"Rute            : {start} -> {end}")
        else:
            print(f"Rute            : {start} -> {end} via {path[1]}")

        print(f"Jarak           : {total_distance} km")
        print(f"Tarif           : Rp {tarif}")
        print(f"Sisa Saldo      : Rp {saldo}")
        print(gate_open)


    def proses_transaksi(self):
        """
        Proses transaksi pembayaran tol
        """
        print("================================ PROGRAM PINTU TOL OTOMATIS ================================")
        print("                                       TOL JORR                                      ")

        while True:
            # Input gerbang masuk dan keluar
            start = input("Masukkan gerbang tol masuk: ").upper()
            end = input("Masukkan gerbang tol keluar: ").upper()

            # Cari jalur terdekat
            jalur_terdekat = self.find_paths_via_neighbors(start, end)

            print(f"\nOpsi jalur dari {start} ke {end}:")
            for i, (path, distance) in enumerate(jalur_terdekat, 1):
                if len(path) < 3:
                    print(f"{i}. Jalur: {start} -> {end}, Jarak Total: {distance} km")
                else:
                    print(f"{i}. Jalur: {start} -> {end} via {path[1]}, Jarak Total: {distance} km")

            # Pilih rute
            while True:
                try:
                    pilihan_rute = int(input("\nRute mana yang ingin anda pilih? (input dengan angka): "))
                    if 1 <= pilihan_rute <= len(jalur_terdekat):
                        # Ambil rute yang dipilih
                        path, total_distance = jalur_terdekat[pilihan_rute - 1]
                        break
                    else:
                        print("Pilihan rute tidak valid. Silakan pilih nomor rute yang tersedia.")
                except ValueError:
                    print("Masukkan harus berupa angka. Silakan coba lagi.")

            # Input detail kendaraan
            jenis_kendaraan = input("Masukkan jenis kendaraan (bus/mobil/pick-up/truk): ").lower()

            # Tentukan golongan
            if jenis_kendaraan in ["bus", "mobil", "pick-up"]:
                golongan = 1
            elif jenis_kendaraan == "truk":
                while True:
                    try:
                        jumlah_gandar = int(input("Masukkan jumlah gandar kendaraan (2-5): "))
                        if 2 <= jumlah_gandar <= 5:
                            golongan = jumlah_gandar
                            break
                        else:
                            print("Jumlah gandar harus antara 2-5.")
                    except ValueError:
                        print("Masukkan harus berupa angka. Silakan coba lagi.")

            # Hitung tarif
            tarif = self.hitung_tarif(total_distance, golongan)

            # Simulasi pembayaran
            while True:
                try:
                    saldo = int(input("Masukkan saldo e-toll: "))
                    break
                except ValueError:
                    print("Masukkan harus berupa angka. Silakan coba lagi.")

            if saldo >= tarif:
                saldo -= tarif
                self.cetak_struk(start, end, path, total_distance, jenis_kendaraan, golongan, tarif, saldo)
            else:
                print("\n═════════════════════════════════ SALDO TIDAK MENCUKUPI ═════════════════════════════════")
                print(f"Tarif           : Rp {tarif}")
                print(f"Saldo Anda      : Rp {saldo}")

                # Tawaran pembayaran alternatif
                lanjut = input("Ingin melanjutkan dengan metode pembayaran lain? (YA/TIDAK): ").upper()

                if lanjut == 'YA':
                    # Proses pembayaran alternatif
                    pembayaran_berhasil = self.pembayaran_alternatif(tarif)

                    if pembayaran_berhasil:
                        self.cetak_struk(start, end, path, total_distance, jenis_kendaraan, golongan, tarif, 0)
                    else:
                        print(gate_close)
                        continue
                else:
                    print(gate_close)
                    continue

            # Loop untuk menunggu mobil melewati gerbang
            while True:
                lewat = input("Apakah mobil telah melewati gerbang tol? (YA/TIDAK): ").upper()
                if lewat == 'YA':
                    print(gate_close)
                    break
                elif lewat == 'TIDAK':
                    continue
                else:
                    # Opsi menonaktifkan gerbang tol
                    print("""
    ╔════════════════════════════════╗
    ║    PINTU TOL TERTUTUP          ║
    ║    Pintu tol tidak aktif       ║
    ╚════════════════════════════════╝
                    """)
                    break
            break

# Menjalankan program utama
if __name__ == "__main__":
    toll_system = TollGateSystem()
    toll_system.proses_transaksi()